In [1]:
import sys
# Import functions from PrismParser (https://github.com/KULL-Centre/PrismData/tree/main)
sys.path.append('/groups/sbinlab/software/PrismData/')

In [2]:
import PrismData
import pandas as pd
import os
import subprocess
import Bio

In [3]:
# Function to read PRISM files as a csv and metadata (commnted out section)
# with information on sequences and column descriptions
def read_from_prism(prismfile):
    parser = PrismData.PrismParser()
    dataframe = parser.read(prismfile).dataframe
    meta_data = parser.read_header(prismfile)
    return meta_data, dataframe

In [4]:
# List of PDBs in SAFES
pdb_list = ['1JTG', '5E9D', '3SZK', '3BN9', '2J0T', '1KTZ', '3SGB', '3MZG', '1DAN', '1VFB', '1OGA', '1PPF', '1AO7', '1CHO', '1R0R', '1MHP', '3HFM', '1KNE', '2JEL', '2FTL', '1IAR']

# Dictionary with interfaces as in SKEMPI 2.0, but with AF2&3 chain names
dict_interface = {'1JTG': 'A_B',
                  '5E9D': 'AB_CDE',
                  '3SZK': 'AB_C',
                  '3BN9': 'A_BC',
                  '2J0T': 'A_B',
                  '1KTZ': 'A_B',
                  '3SGB': 'A_B',
                  '3MZG': 'A_B',
                  '1DAN': 'AB_CD',
                  '1VFB': 'AB_C',
                  '1OGA': 'ABC_DE',
                  '1PPF': 'A_B',
                  '1AO7': 'ABC_DE',
                  '1CHO': 'ABC_D',
                  '1R0R': 'A_B',
                  '1MHP': 'A_BC',
                  '3HFM': 'AB_C',
                  '1KNE': 'A_B',
                  '2JEL': 'AB_C',
                  '2FTL': 'A_B',
                  '1IAR': 'A_B'}

In [5]:
# Original 7085 entry SKEMPI 2.0 from the website (https://life.bsc.es/pid/skempi2)
skempi2_raw = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempi_v2.csv', sep=';')

# Filtered dataset (filtering documented in SKEMPIv2_filtering_documented.ipynb)
skempi2 = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempi2_final_final.csv')

In [6]:
# Create a dictionary for variant counts - to keep track of any lost datapoints
count_dict = dict()

# Define paths for inputs - AF runs and xtal run 
xtal_path = '/groups/sbinlab/panf/SKEMPIv2_benchmark/cartesian_runs_folder/'
af2_path = '/groups/sbinlab/zqt390/Thesis_Project/af2_files/'
af3_path = '/groups/sbinlab/zqt390/Thesis_Project/af3_files/'

# Where to place PRISM files for further processing (merging)
output_folder = '/groups/sbinlab/panf/SKEMPIv2_benchmark/SAFES_prisms'

# Create a dictionary for sequences - to keep track of differences
# between AF structure sequences and xtal
dict_seq = dict()

# Go through all of the SAFES PDBs
for pdb_id in pdb_list:
    print(pdb_id) # To keep track of what's loaded if any errors are raised
    dict_seq[pdb_id] = dict() # Sub dict for every PDB sequences

    # Construct file paths
    
    # Default cartesian pipeline output PRISM file path
    xtal = os.path.join(xtal_path, pdb_id, 'output', f'prism_rosetta_XXX_{pdb_id}.txt')
    # Output of InterfaceAnalyzer
    xtal_IA = os.path.join(xtal_path, pdb_id, 'output', f'{pdb_id}_full_interface_IAM_single_load.csv')

    af2 = os.path.join(af2_path, f'output_{pdb_id}', 'output', f'prism_rosetta_XXX_{pdb_id}_af2.txt')
    # af2_IA = os.path.join(af2_path, f'output_{pdb_id}', 'output', f'{pdb_id}_IAM.csv')
    af2_IA = os.path.join(af2_path, f'output_{pdb_id}', 'output', f'{pdb_id}_incf_fixed.csv')
    
    af3 = os.path.join(af3_path, f'output_{pdb_id}', 'output', f'prism_rosetta_XXX_{pdb_id}_af3.txt')
    # af3_IA = os.path.join(af3_path, f'output_{pdb_id}', 'output', f'{pdb_id}_IAM.csv')
    af3_IA = os.path.join(af3_path, f'output_{pdb_id}', 'output', f'{pdb_id}_IAM_incf_fixed.csv')

    # Define pairs of paths to take PRISM metadata from the original pipeline output and 
    # make a PRISM file from a IA csv
    path_pairs = {'xtal': (xtal, xtal_IA),
                  'af2': (af2, af2_IA),
                  'af3': (af3, af3_IA)}

    # Write IA.csvs as prism files using the prism headers from cartesian runs
    count_dict[pdb_id] = dict()
    
    # Add the number of variant for this PDB ID in SKEMPI 2.0 (multi+single and single)
    # to the variant count dictionary
    count_dict[pdb_id]['sk2_len_all'] = len(skempi2.loc[skempi2['PDB_ID']==pdb_id])
    count_dict[pdb_id]['sk2_len_single'] = len(skempi2.loc[
            (skempi2['Mutation(s)_PDB'].str.split(',').str.len()==1)&(skempi2['PDB_ID']==pdb_id)])
    
    for pair in path_pairs.keys():

        # Read pipeline output prism file's metadata
        metadata = read_from_prism(path_pairs[pair][0])[0]

        # Save the sequence from the metadata
        dict_seq[pdb_id][pair] = metadata['protein']['sequence']

        # Add info about two new columns (InterfaceAnalyzer scores and stds)
        metadata['columns']['mean_ddG_infc'] = 'mean InterfaceAnalyzer ddG values (mean((MUT-WT)/2.9) kcal/mol)'
        metadata['columns']['std_ddG_infc'] = 'std InterfaceAnalyzer ddG values (std((MUT-WT)/2.9) kcal/mol)'

        # Construct an output file name and add it to the metadata
        metadata['filename'] = f'prism_rosetta_XXX_{pdb_id}_cartIA_{pair}.txt'

        # Read the classic pipeline + IA csv
        try:
            ia = pd.read_csv(path_pairs[pair][1])
            # Count the number of non-wild type single variants 
            count_dict[pdb_id][pair] = len(ia.loc[(~ia['variant'].str.endswith('='))&(ia['variant'].str.split(':').str.len()==1)])
        except:
            print('Cannot read the IA csv')
            continue

        # Make a PRISM file from a IA csv
        prismdata = PrismData.VariantData(metadata=metadata, dataframe=ia)
        prismdata.check()
        pp = PrismData.PrismParser()
        pp.write(os.path.join(output_folder, metadata['filename']), prismdata)

1JTG
5E9D
3SZK
3BN9
2J0T
1KTZ
3SGB
3MZG
1DAN
1VFB
1OGA
1PPF
1AO7
1CHO
1R0R
1MHP
3HFM
1KNE
2JEL
2FTL
1IAR


In [ ]:
# Run PRISM parser

for pdb_id in pdb_list:
    subprocess.run(["python3", "/groups/sbinlab/software/PrismData/PrismData.py", "--merge", os.path.join(output_folder, 'merged_infc_fixed', f"prism_rosetta_XXX_{pdb_id}_xtal_af2_3.txt"), os.path.join(output_folder, f"prism_rosetta_XXX_{pdb_id}_cartIA_xtal.txt"), os.path.join(output_folder, f"prism_rosetta_XXX_{pdb_id}_cartIA_af2.txt"), os.path.join(output_folder, f"prism_rosetta_XXX_{pdb_id}_cartIA_af3.txt")])

In [7]:
# Rename the colunms to meaningful names and concat the dataframes into one

safes_all = pd.DataFrame()
for file in os.listdir(os.path.join(output_folder, 'merged_infc_fixed')):
    if not file == '.ipynb_checkpoints':
        pdb_name = file.split('_')[3]
        df = pd.read_csv(os.path.join(output_folder, 'merged_infc_fixed', file), sep='\s+', comment='#')
        df.rename(columns={'mean_ddG_00': "mean_ddG_xtal", 'std_ddG_00': "std_ddG_xtal",
                   'mean_ddG_infc_00': "mean_ddG_infc_xtal", 'std_ddG_infc_00':'std_ddG_infc_xtal',
                   'mean_ddG_01': 'mean_ddG_af2', 'std_ddG_01': 'std_ddG_af2',
                   'mean_ddG_infc_01': 'mean_ddG_infc_af2', 'std_ddG_infc_01':'std_ddG_infc_af2',
                   'mean_ddG_02': 'mean_ddG_af3', 'std_ddG_02': 'std_ddG_af3',
                   'mean_ddG_infc_02': 'mean_ddG_infc_af3', 'std_ddG_infc_02':'std_ddG_infc_af3'},
          errors="raise", inplace=True)
        df.insert(0, 'PDB_ID', pdb_name) # Add PDB ID

        # Count the variants in the merged PRISM file
        count_dict[pdb_name]['merged'] = len(df.loc[(df['variant'].str.split(':').str.len()==1)&(~df['variant'].str.endswith('='))])
        
        safes_all = pd.concat([safes_all, df])

In [8]:
# Extract single mutants that aren't wild type
safes_sing = safes_all.loc[(safes_all['variant'].str.split(':').str.len()==1)&(~safes_all['variant'].str.endswith('='))].copy()

In [12]:
safes_sing

,PDB_ID,variant,mean_ddG_xtal,std_ddG_xtal,mean_ddG_infc_xtal,std_ddG_infc_xtal,mean_ddG_af2,std_ddG_af2,mean_ddG_infc_af2,std_ddG_infc_af2,mean_ddG_af3,std_ddG_af3,mean_ddG_infc_af3,std_ddG_infc_af3,wt_pos
1,1MHP,F283W,0.976092,3.036746e-03,0.062596,6.406080e-03,1.105977,0.000000,-0.033537,7.392192e-14,2.077701,1.262590e-02,0.947402,5.864598e-02,F283
2,1MHP,F283Y,0.353218,2.231943e-01,-0.131792,3.886540e-01,0.258621,0.002890,-0.000185,4.085368e-04,0.611724,2.600853e-03,0.429577,9.195515e-03,F283
6,1MHP,G237A,0.298276,0.000000e+00,-0.237788,6.825092e-03,-0.392069,0.000975,0.384370,2.303085e-03,-1.241379,0.000000e+00,-0.613928,0.000000e+00,G237
7,1MHP,G237N,1.604483,2.220446e-16,0.218363,6.825092e-03,2.397816,0.000650,1.999644,3.380392e-03,2.191379,0.000000e+00,1.596371,6.135398e-12,G237
8,1MHP,G237Q,0.930000,0.000000e+00,1.035314,2.382143e-03,2.256322,0.000163,2.614952,2.765655e-03,1.056897,0.000000e+00,0.181126,5.987574e-12,G237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,3SZK,T320A,1.529655,2.220446e-16,0.097085,1.387779e-17,1.367586,0.000000,0.093444,1.955620e-03,1.383908,2.220446e-16,-0.055904,0.000000e+00,T320
35,3SZK,T352A,1.666897,9.753197e-04,1.243816,1.860906e-03,0.511264,0.029585,-0.166909,6.816185e-03,1.427931,0.000000e+00,1.346679,0.000000e+00,T352
37,3SZK,V309A,2.418966,0.000000e+00,0.003076,2.133620e-03,2.682184,0.000000,-0.003369,1.812221e-03,2.768966,4.440892e-16,-0.304561,0.000000e+00,V309
39,3SZK,Y322A,3.223793,1.462980e-03,1.625647,1.632063e-02,3.875747,0.451821,2.023185,1.549575e-03,2.724828,0.000000e+00,1.163914,0.000000e+00,Y322


In [11]:
safes_sing['wt_pos'] = safes_sing['variant'].str[:-1]

In [13]:
safes_wt = safes_all.loc[(safes_all['variant'].str.split(':').str.len()==1)&(safes_all['variant'].str.endswith('='))]

,PDB_ID,variant,mean_ddG_xtal,std_ddG_xtal,mean_ddG_infc_xtal,std_ddG_infc_xtal,mean_ddG_af2,std_ddG_af2,mean_ddG_infc_af2,std_ddG_infc_af2,mean_ddG_af3,std_ddG_af3,mean_ddG_infc_af3,std_ddG_infc_af3
0,1MHP,F283=,1.045391e-13,0.076699,NaN,NaN,-5.226982e-14,0.001626,-0.005213,4.083437e-04,-5.227069e-14,0.032408,0.008968,3.701530e-03
5,1MHP,G237=,0.000000e+00,0.000000,NaN,NaN,0.000000e+00,0.000000,0.000718,3.696034e-14,0.000000e+00,0.000000,0.020975,6.283258e-12
11,1MHP,G238=,0.000000e+00,0.000000,NaN,NaN,-1.045414e-13,0.167952,0.001413,1.102227e-03,0.000000e+00,0.000000,-0.003503,0.000000e+00
17,1MHP,G284=,0.000000e+00,0.000000,NaN,NaN,3.614007e-20,0.001270,-0.005301,7.274919e-04,1.045379e-13,0.032267,-0.033079,2.945741e-03
24,1MHP,G286=,-5.227069e-14,0.040963,NaN,NaN,0.000000e+00,0.000000,0.005649,0.000000e+00,0.000000e+00,0.032974,-0.058728,2.747779e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,3SZK,T320=,0.000000e+00,0.000000,NaN,NaN,0.000000e+00,0.000000,-0.000385,1.740823e-03,5.226982e-14,0.001832,-0.074096,2.104078e-03
34,3SZK,T352=,0.000000e+00,0.000000,NaN,NaN,5.227069e-14,0.034299,-0.111371,3.761848e-03,0.000000e+00,0.000000,0.005876,3.500338e-03
36,3SZK,V309=,0.000000e+00,0.000000,NaN,NaN,-1.807004e-20,0.000905,-0.002703,1.766325e-03,0.000000e+00,0.000000,-0.297079,0.000000e+00
38,3SZK,Y322=,0.000000e+00,0.000000,NaN,NaN,-5.227300e-14,0.443463,-0.060515,6.255485e-04,-1.568095e-13,0.000975,-0.117550,3.766015e-03


In [ ]:
safes_sing.to_csv('SAFES.csv', index=False)

In [20]:
# Check for which PDBs number of variants in merged PRISM is different from 
# the variants in xtal PRISM

for pdb_id in pdb_list:
    diff = count_dict[pdb_id]['xtal'] - count_dict[pdb_id]['merged']
    if diff > 0:
        print('Missing vars in merged: ', pdb_id, diff)
    elif diff < 0:
        print('Missing vars in xtal: ', pdb_id, diff)
    else:
        print('All good!', pdb_id)

Missing vars:  1JTG -2
All good! 5E9D
All good! 3SZK
All good! 3BN9
All good! 2J0T
All good! 1KTZ
All good! 3SGB
All good! 3MZG
Missing vars:  1DAN -1
All good! 1VFB
All good! 1OGA
All good! 1PPF
Missing vars:  1AO7 -1
Missing vars:  1CHO -1
All good! 1R0R
All good! 1MHP
All good! 3HFM
All good! 1KNE
All good! 2JEL
All good! 2FTL
All good! 1IAR


In [10]:
# Identify PDB IDs for wich manual checks might be needed
# (e. g. order of chains is flipped)

import Bio
from Bio import SeqRecord,SeqIO
from Bio.Seq import Seq
from Bio.Align import PairwiseAligner
from Bio import PDB

aligner = PairwiseAligner()
aligner.mode='global'

bad_al = list()

for pdb in dict_seq.keys():
    
    alignments_af = aligner.align(dict_seq[pdb]['af2'], dict_seq[pdb]['af3'])

    if len(dict_seq[pdb]['af2'])!=len(dict_seq[pdb]['af3']):
        for alignment in alignments_af:
            print('AF2 and 3 are different: ')
            print(alignment)

    for alignment in alignments_af:
        if alignment.score != len(dict_seq[pdb]['af2']):
            print('AF2 and 3 are different: ')
            print(alignment)

    alignments_xtal = aligner.align(dict_seq[pdb]['xtal'], dict_seq[pdb]['af2'])
    
    if abs(alignments_xtal[0].score - len(dict_seq[pdb]['af2']))>20:
        if not pdb in bad_al:
            bad_al.append(pdb)

print('Potentially problematic alignments: ')
print(bad_al)